In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForImageTextToText, GenerationConfig, pipeline
import torch
import torch.nn as nn

model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
).to(device="cpu")
model.eval()

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

#prompt = "Tomorrow is the place for all good zombies to come to the aid"
prompt = "Complete successive parts of a sentence given one word at a time:"
target = "Now is the time for all good men to come to the aid of their country ."
targetVec = target.split()

for word in targetVec:
    prompt += ' '
    prompt += word
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device) 
    with torch.no_grad():
        raw_outputs = model(**inputs)
    prob = torch.softmax(raw_outputs.logits, dim=-1)

####################################################
# Muhammad's code adds---put after computing raw outputs

    max = torch.max(raw_outputs.logits[0, -1, :])
    exp1 = torch.exp(raw_outputs.logits[0, -1, :]-max)   # deltas
    exp2 = torch.exp(exp1) - 1
    double_sf_probs = exp2 / torch.sum(exp2)
    _, top_id = torch.topk(prob[0, -1,:], 50)

    print("Single,doube softmaxed probs from raw output logits on deltas for prompt: ", prompt)

    for t_id in top_id:
        t_str = tokenizer.decode(t_id.item()).strip()
        #print(f"{t_str}: {raw_outputs.logits[0, -1, :][t_id].item()}; {prob[0, -1,:][t_id].item():.4f}; {double_sf_probs[t_id].item():.4f}")
        print(f"{t_str}: {raw_outputs.logits[0, -1, :][t_id].item()}")
    
    print("Press Enter to continue...")
    input()
    


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Single,doube softmaxed probs from raw output logits on deltas for prompt:  Complete successive parts of a sentence given one word at a time: Now
,: 20.25
I: 16.625
you: 16.5
the: 16.375
is: 15.6875
suppose: 15.375
that: 15.25
consider: 14.8125
: 14.5625
we: 14.5625
let: 14.25
think: 14.0625
it: 13.9375
imagine: 13.875
: 13.75
: 13.75
,: 13.625
choose: 13.5
.: 13.375
.: 13.3125
write: 13.25
a: 13.1875
in: 13.0625
...: 13.0625
...: 13.0
...: 12.9375
to: 12.9375
your: 12.875
You: 12.8125
complete: 12.8125
for: 12.75
what: 12.75
:: 12.625
assume: 12.625
,: 12.625
here: 12.5625
find: 12.5
.: 12.375
: 12.375
...: 12.3125
,...: 12.125
...: 12.0625
each: 12.0
(: 12.0
here: 12.0
compute: 11.9375
i: 11.9375
[: 11.9375
divide: 11.875
proceed: 11.875
Press Enter to continue...
Single,doube softmaxed probs from raw output logits on deltas for prompt:  Complete successive parts of a sentence given one word at a time: Now is
the: 15.0
a: 14.5
now: 13.625
[: 13.375
(: 12.6875
: 12.625
\: 12.375
Now: 1

KeyboardInterrupt: Interrupted by user